# Clase 2 - Anomaly Detection

En este notebook llevaremos a cabo los ejemplos para cada uno de los modelos que revisamos durante la clase número 2, dedicada a detección de anomalías.

# Isolation Forest

El primer modelo que revisaremos es Isolation Forest, que es un modelo ensamblado de árboles para detectar observaciones anómalas.
Para este ejemplo, trabajaremos con la base de datos cardio.mat, base ampliamente conocida, donde las observaciones ya se encuentran etiquetados como anomalías o no (algo que típicamente no ocurre, la etiqueta de anomalía no se tiene, pero es bueno tenerla para efectos pedagogicos)

Para poder realizar el Isolation Forest en Python, es necesario importar la función `IsolationForest` del modulo `sklearn`.

In [5]:
pip install mat4py

  Obtaining dependency information for mat4py from https://files.pythonhosted.org/packages/55/e9/4e2deb2c904e1d0bde3fa49c054160c2f56ffee5d327dc65fe41bb360613/mat4py-0.6.0-py2.py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import numpy as np
import pandas as pd
from mat4py import loadmat #para cargar los datos en formato .mat
from sklearn.ensemble import IsolationForest

#cargamos y formatamos los datos
cardio_data = loadmat(filename='./Data/cardio.mat')
cardio_data_x = pd.DataFrame(cardio_data['X'])
cardio_data_x.columns = ['col_' + str(i) for i in cardio_data_x.columns]
cardio_data_y = pd.DataFrame(cardio_data['y'])
cardio_data_y = cardio_data_y.to_numpy().flatten()

cardio_data_x.head()

FileNotFoundError: [Errno 2] No such file or directory: './Data/cardio.mat'

In [ ]:
cardio_data_y

In [ ]:
len(cardio_data_y)

In [ ]:
sum(cardio_data_y)

In [ ]:
cardio_data_x.shape

ya con los datos más formateados (no pude conseguir los nombres de las variables lamentablemente :'(), podemos ajustar el modelo de isolation forest. Ahora, cuando ajustamos K-means y sus amigos, solo debiamos entregar el número de clusters, lo cual era natural, pero en el caso del isolation forest que es un modelo ensamblado de árboles, es necesario entregarle ciertos `hiperparámetros` para que funcione. Estos `hiperparámetros` ayudan a guiar al modelo en su entrenamiento, permitiendo mejorar (o emperorar) el modelo según los vamos haciendo variar.

In [7]:
IsoFor_model = IsolationForest(
    n_estimators = 500, #Numero de árboles considerados
    max_samples = 'auto', #Numero de observaciones para entrenar
    n_jobs=-1,
    contamination = 0.2, #Porcentaje de anomalías esperada
    random_state = 2023 #Semilla para reproducibilidad
)
IsoFor_model

IsolationForest(contamination=0.2, n_estimators=500, n_jobs=-1,
                random_state=2023)

Dentro de los hiperparámetros antes definidos el de `contamination` es muy importante, dado que significa que una vez que tengamos el score de anomalía entregado por el modelo, entonces nos quedaremos con el percentil 1 (1%) inferior como observaciones anómalas.

Aquí para tenerlo en cuenta, los scores de anomalía quedan acotados entre -1 y 0, donde mientras más próximo a -1 sea el valor significa más evidencia de anomalía, mientras que hacia el 0 se espera un valor normal.

Obtengamos dichos scores

In [8]:
IsoFor_model.fit(cardio_data_x)
anomaly_score = IsoFor_model.score_samples(cardio_data_x)
len(anomaly_score)

NameError: name 'cardio_data_x' is not defined

In [ ]:
anomaly_score

lo anterior son los scores de anomalía que son utiles en el caso que uno quiera manipular las anomalías de alguna manera. Si queremos utilizar directamente el parámetro de contaminación para saber cuales son y cuales no son anomalías, se hace de la siguiente forma

In [3]:
anomaly_classification = IsoFor_model.predict(cardio_data_x)
anomaly_classification #aquí 1 es normal y -1 es anomalía

NameError: name 'IsoFor_model' is not defined

In [2]:
# comparamos con y
print(anomaly_classification == -1)

NameError: name 'anomaly_classification' is not defined

es interesante ver como es la distribución de los scores de anomalía, dado que esperaríamos tener una gran masa hacia la derecha de 0.5, porque la gran mayoría de observaciones son normales. Revisemos eso

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

percentil_01 = np.quantile(anomaly_score, q = 0.1)

fig, ax = plt.subplots(figsize = (15,8))
sns.distplot(
    anomaly_score,
    hist = False,
    rug = True,
    color = 'blue',
    kde_kws = {'shade':True, 'linewidth':1},
    ax = ax
)
ax.axvline(percentil_01, c = 'red', linestyle = '--', label = 'Percentil 1%')
ax.set_xlabel('Anomaly Scores')
ax.set_ylabel('Density')


finalmente, podemos comparar como le fue al modelo, dado que conocemos las etiquetas de si las observaciones son anomalías o no

In [ ]:
class_pred = pd.DataFrame(anomaly_classification)
class_pred.columns = ['Model']
real_class = pd.DataFrame(cardio_data_y)
real_class.columns = ['Real']
anomaly_check = pd.concat([class_pred,real_class],axis = 1)
print(anomaly_check.query('Model == -1')['Real'].sum())
print(sum(cardio_data_y))

In [ ]:
print(141/366*100)
print(141/176*100)

## Support Vector Machine

Aplicaremos un modelo especial de support vector machine que es el one-class support vector machine, para los mismos datos anteriores y así compararemos la performance del modelo anterio respecto a este.

In [ ]:
from sklearn.svm import OneClassSVM

In [ ]:
one_class_model = OneClassSVM(
    nu = 0.2, #proporcion de datos anomalos
    kernel = 'rbf', #Kernel
    gamma = 'auto' #parametros asociado al kernel
)
one_class_model.fit(cardio_data_x)

ahora hacemos las predicciones para saber que observaciones está considerando como anómalas y cuales no

In [ ]:
svm_anomalies = one_class_model.predict(cardio_data_x)
svm_anomalies

In [ ]:
print(sum(svm_anomalies == -1))
print(sum(svm_anomalies == 1))

ahora comparamos!

In [ ]:
class_pred_svm = pd.DataFrame(svm_anomalies)
class_pred_svm.columns = ['Model_svm']
print(class_pred_svm.query('Model_svm == -1').count())
anomaly_svm_check = pd.concat([class_pred_svm,real_class],axis = 1)
print(anomaly_svm_check.query('Model_svm == -1')['Real'].sum())

In [ ]:
print(126/366*100)
print(126/176*100)

entre modelos

In [ ]:
class_pred

In [ ]:
cross_anomaly_check

In [ ]:
cross_anomaly_check.query('check == -2')['check']

In [ ]:
cross_anomaly_check = pd.concat([class_pred,class_pred_svm],axis = 1)
cross_anomaly_check['check'] = cross_anomaly_check.sum(axis = 1)
cross_anomaly_check.query('check == -2')['check'].count()
#print(anomaly_svm_check.query('Model_svm == -1 and Model == -1')['Real'].sum())

In [ ]:
cross_anomaly_check_2 = pd.concat([cross_anomaly_check,real_class],axis = 1)
print(cross_anomaly_check_2.query('check == -2')['Real'].sum())

## PCA 

Finalmente, nuevamente recurrimos a la descomposición en componentes principales para, esta vez, hacer detección de anomalías

Lo primero que debeos hacer es cargar los módulos correspondientes y también estandarizar los datos para poder hacer la descomposición

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaled_cardio_data = pd.DataFrame(scaler.fit_transform(cardio_data_x), columns=cardio_data_x.columns)
scaled_cardio_data.head()

ahora, realizamos la descomposición en componentes principales

In [ ]:
pca_model = PCA()
pca_model.fit(scaled_cardio_data)

In [ ]:
plt.plot(range(1,22),pca_model.explained_variance_ratio_.cumsum(), marker = 'o')
plt.xlabel('Número de componentes')
plt.ylabel('Variabilidad explicada acumulada')

viendo la gráfica, haremos un corte en 80% de variabilidad explicada, lo que corresponde a considerar 8 componentes

In [ ]:
pca_model.explained_variance_ratio_.cumsum()

In [ ]:
pca_model = PCA(n_components = 8)
pca_model.fit(scaled_cardio_data)
transformed_cardio_data = pd.DataFrame(pca_model.transform(scaled_cardio_data))
transformed_cardio_data

ahora, lo que debemos hacer es calcular el error de transformación... para esto debemos reconstruir nuestros datos a partir de considerar solo 8 componentes principales

In [ ]:
restored_cardio_df = pd.DataFrame(pca_model.inverse_transform(transformed_cardio_data), columns = scaled_cardio_data.columns)
restored_cardio_df.head()

In [ ]:
scaled_cardio_data.head()

In [ ]:
def get_anomaly_scores(df_original, df_restored):
    loss = np.sum((np.array(df_original) - np.array(df_restored)) ** 2, axis=1)
    loss = pd.Series(data=loss, index=df_original.index)
    return loss

def is_anomaly(data, pca_model, threshold):
    pca_data = pca_model.transform(data)
    restored_data = pca_model.inverse_transform(pca_data)
    loss = np.sum((data - restored_data) ** 2)
    return loss > threshold

In [ ]:
reconstruction_error = get_anomaly_scores(scaled_cardio_data,restored_cardio_df)
reconstruction_error

graficando

In [ ]:
plt.figure(figsize=(16,8))
sns.lineplot(data = reconstruction_error)

In [ ]:
plt.figure(figsize=(16,8))
sns.lineplot(data = reconstruction_error)
for index, row in scaled_cardio_data.iterrows():
    if is_anomaly([row],pca_model,20):
        plt.axvline(row.name, color = 'red', alpha = 0.2)

In [ ]:
anomaly_data = pd.DataFrame(get_anomaly_scores(scaled_cardio_data,restored_cardio_df))
anomaly_data.columns = ['Score']
anomaly_data.query('Score >= 10').count()  #Fira-code font

¿a cuánto le achuntó?

In [ ]:
anomaly_data_pca = anomaly_data
anomaly_data_pca['Model_pca'] = np.where(anomaly_data['Score'] >= 10, -1, 1)
anomaly_pca_check = pd.concat([anomaly_data_pca['Model_pca'],real_class],axis = 1)
print(anomaly_pca_check.query('Model_pca == -1')['Real'].sum())

In [ ]:
print(47/138*100) #Anomalias real detectadas sobre el total de detectadas
print(47/176*100) #Anomalias real detectadas sobre el total de reales

comparando con Isolation Forest

In [ ]:
anomaly_data_pca = anomaly_data
anomaly_data_pca['Model_pca'] = np.where(anomaly_data['Score'] >= 10, -1, 1)
anomaly_data_pca

In [ ]:
cross_anomaly_check = pd.concat([class_pred,anomaly_data_pca['Model_pca']],axis = 1)
cross_anomaly_check['check'] = cross_anomaly_check.sum(axis = 1)
cross_anomaly_check.query('check == -2')['check'].count()

y con SVM

In [ ]:
cross_anomaly_check = pd.concat([anomaly_data_pca['Model_pca'],class_pred_svm],axis = 1)
cross_anomaly_check['check'] = cross_anomaly_check.sum(axis = 1)
cross_anomaly_check.query('check == -2')['check'].count()